# Apache Spark
Sistema diseñado para procesar datos de manera distribuida sobre clusters. Spark puede procesar cantidades de datos en el orden de terabytes incluso petabytes. El concepto de spark es imaginar el cluster como una memoria gigante, la memoria resltante de combinar las memorias de todos los clusters. Se prioriza el uso de memoria y consigue ser muy rapido el procesamirento de la informacion, mayor que si seutilizara MapReduce (Google). Spark utiliza MapReduce para algunas tareas de clasificacion mediante regresion. Apache Spark esta implementado en Scala que es ejecutado en la maquina virtual de java. Ademas de Spark ofrece interfaces de programacion para java, python y R.

Caracteristicas: velocidad de procesaiento, soporte multilenguajes, analisis avanzado. Se puede desarrollar en tres maneras:

1) Solo: standalone. Utiliza como base HDFS y encima se encuentra spark
2) Hadoop (Yam, administrador de recursos): Base HDFS -> Yarn/Mesos(kernel administrador del cluster) ->Spark
3) Spark con MapReduce (Spark In MapReduce, SIMR): Base HDFS -> MapReduce y dentro se encuentra Spark



Cuando se ejecuta spark en un ambiente distribuida se distinguen dos tipos de procesos: driver y executor Un proceso 
driver conectado a 3 procesos


driver: proceso principal. Este proceso tiene un objeto SparkContext que le permite conectar con el gestor del cluster  y reservar proceso executor en los distintos nodos del cluster. Cada uno de los nodos del cluster (tambien se conocen como worker) podra ejecutar uno o varios procesos executor que almacenara fragmentos de los datos del programa y realizara operaciones sobre ellos. Durante la ejecucion ira enviando peticiones a los distintos procesos executor que pueden contactar entre ellos para realizar tareas y comunicar con el proceso driver para devolver resultados.

RDD (Resilient Distributed Datasets) Tipo de datos basico. Estos datos almacenan informacion de manera distribuida entre todos los equipos del cluster. Durante la ejecucion de un programa Spark se construyen varios RDDs que se dividen en distintos fragmentos y son almacenados en la memoria de los equipos del cluster. Caracteristica:

    1) Estan formados por un conjunto de registros, tambien llamados elementos, todos son del mismo tipo . Por ejemplo si cargamos un RDD apartir de un archivo plano se creara un RDD de cadenas de texto, una por cada linea del archivo. En Scala o Java, al declarar  un RDD se debe definir el tipo de los registros: Lenguajes estaticos : RDD[String] en Scala y JavaRDD en Java.
    En dinamicos: python se pueden mezclar los tipos de datos.
    
    RDDs han sido diseñados desde el inicio para ser distribuidos: los registros que lo componen se repartiran entre los cluster. Para realizar esta distribucion: los RDDs se dividen en particiones. Cada particon se almacena unicamente en un proceso executor dentro de un nodo del cluster, aunque un proceso executor puede albergar distintas peticiones de distintos RDDs. El numero de particiones em la que dividir un RDD se puede configurar e incluso cambiar a lo largo de la ejecucion por default es el numero de nucleos de procesamiento disponibles en el cluster.
    
    Para decidir que registros forman parte de cada particion, Spark utiliza particionadores, que son funciones que toman un registro y devuelven el numero de la particion a la que pertenecen. Estos particionadores se puede configurar si se desea mejorar el rendimiento o dejar el default
    
    Ejemplo como se podria particionar un RDD de 13 parejas (int, str) sobre 3 procesos executor utilizando el rango de valores del primer elemento de la pareja.
    
    
    Los RDDs son inmutables, no se puede modificar ni actualizar. Una vezx creado asi permacece hasta que termine la ejecucion del programa

In [2]:
#SparkContext : metodo inicial para trabajar con PySpark

In [1]:
#conda install -c conda-forge findspark
import findspark
findspark.init()
findspark.find()

'C:\\BigData\\Spark\\Spark'

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext(appName="Prueba")
#sc objeto que apunta a SC al cluster local

In [6]:
sc.stop()

In [4]:
type(sc)

pyspark.context.SparkContext

In [5]:
sc.master

'local[*]'

In [6]:
sc.appName

'Prueba'

In [7]:
import random

In [8]:
#Creando un RDD de 5 enteros: se crea en la memoria del proceso driver
r = sc.parallelize([1,2,3,4,5])
type(r)


pyspark.rdd.RDD

In [15]:
print(r)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274


In [9]:
r.collect()

[1, 2, 3, 4, 5]

In [6]:
#Creando un RDD de 3 cadenas de texto
r = sc.parallelize(["hola", "hi", "ciao"])
type(r)

pyspark.rdd.RDD

In [7]:
r.collect()

['hola', 'hi', 'ciao']

In [8]:
#Al crear RDDs a partr de colecciones en memoria del proceso driver
#estamos limitados a la memoria que este proceso tenga disponible.
#Por eso, es mejor generar RDDs a partir de archivos.
#Spark propotcion el metodo textfile, que carga el arch de texto y
#genera n RDD de cadenas de texto
#en un sistema distribuido:
#"hdfs://node:port/data/file.txt, Amazon S3 "s3n://bucket/file.txt"
#Spark cuenta con metodos para abrir archivos en fotmato
#hadoop sc.hadoopFile
r = sc.textFile("C:\\DataSet\\titanic.csv")
print(r)

C:\DataSet\titanic.csv MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0


In [10]:
print(r.collect,"\n")

<bound method RDD.collect of C:\DataSet\titanic.csv MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0> 



In [11]:
#Acciones (se ejecutan inmediatamente en todo el cluster)
#son operaciones que realizan un procesamiento
#sobre todo un RDD y devuelve un valor, dejando el RDD original
#en el mismo estado (es inmutable).
#El valor generado se envia al proceso driver
#Un RDD puede almacenar gigabytes, terabytes o incluso
#pentabytes de datos de manera distribuida pero el valor generado
#por las acciones se regresara al proceso driver. En este caso,
#tomar en cuenta la memoria del proceso driver
#las acciones se ejecutan de manera inmediata a diferencia de 
#Las transformaciones
#Metodo collect:recorre el RDD y devuelve una lista RDDS pequeños
r = sc.parallelize([1,2,3,4,5,6])
print(type(r.collect()))
r.collect()
#se generaron 6 elementos en el cluster, collect busca los elementos
#en el cluster
#y devuelve la informacion al proceso driver

<class 'list'>


[1, 2, 3, 4, 5, 6]

In [13]:
#Para muchos elementos en el cluster se utiliza take
#que se devuelve los primeros elementos encontrados al driver,
#no muestran todos los elementos
#sc.setLogLevel("WARN")
r = sc.parallelize(range(100))
r.take(100)
#sc.parallelize(range(1000)).cache().take(5)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [14]:
#Cuenta los elementos del RDD
r.count()

100

Una de las acciones mas utiles sobre los RDD es: reduce permite recorrer todos los valores de un RDD y calcular un valor en relacion con ellos. Reduce un RDD a un unico valor. Dado un RDD con elementos de tipo T, la funcion de reduccion es una funcion binaria que acepta dos elementos de tipo T y devuelve un valor del mismo tipo T, tiene tipo T x T ->T.
reduce(f) es un metodo que aplica la funcion de reduccion f a los 2 primeros elementos. Luego aplica de nuevo la funcion f al valor resultante y al tercer elemento, y asi sucesivamente hasta que procesa el ultimo elemento y produce el valor final.
Ejemplo: Se muestra el proceso para calcular la suma de un RDD de 5 elementos enteros.

In [15]:
#Para crear un RDD con los numeros del 1 al 5 y calcular su suma,
#definiriamos una funcion de reduccion
#add e invocariamos a reduce como sigue:
def add(x, y):
    return x + y

#llamada a reduce
r = sc.parallelize(range(1,6))
r.reduce(add)

15

In [16]:
#Utilizando una funcion anonima o lambda
r = sc.parallelize(range(1,6))
r.reduce(lambda x,y: x+y)

15

In [4]:
#Metodo takeOrdered, take()
r = sc.parallelize(range(1000))
r.takeOrdered(10,lambda x : -x)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 0.0 failed 1 times, most recent failure: Lost task 9.0 in stage 0.0 (TID 9) (10.200.84.217 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spa

In [5]:
r = sc.parallelize(range(1000))
r.takeSample(False,10)#False, True

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 1.0 failed 1 times, most recent failure: Lost task 7.0 in stage 1.0 (TID 19) (10.200.84.217 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.sp

In [6]:
def multiply_positive(x,y):
    if x > 0 and y > 0:
        return x*y
    elif x > 0:
        return x
    elif y > 0:
        return y
    else:
        return 1
    
r = sc.parallelize([-1,2,1,-5,8])  
r.reduce(multiply_positive)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 24) (10.200.84.217 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.sp

In [7]:
r = sc.parallelize(range(3),1)#Una particion [0,1,2]
r.reduce(lambda x,y: x-y)
#(0-1) - 2

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 36) (10.200.84.217 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.sp

In [10]:
r = sc.parallelize(range(3),2)#Dos particiones [0] y [1,2]
r.reduce(lambda x,y: x-y)
#  (0) y (1-2)
# 0       -1
#0     - (-1)

1

In [11]:
#Manejo de cadenas. Utilizacion del metodo aggregate
#Tres parametros:
#1) Un valor inicial para el acumulador zeroValue que tendra tipo C
#2) Una funcion seqOp para combinar elementos de nuestra RDD (de tipo T)
#3) Una funcion combOp para combinar dos acumuladores de tipo C
# y devolver un valor de tipo C
# Cuentas las h en el RDD
r = sc.parallelize(["hola" ,"hi", "ciao"])
#(int, cuentas las "h" suma los acumuladores)
#                                           => (Valor_inicial, map , reduce)
#                  MAP                         Reduce
r.aggregate(0, lambda c,s : c + s.count("h"), lambda c1, c2: c1 + c2)
#(h,1)
#(h,2)
#(h,0)

2

In [13]:
r = sc.parallelize(range(1000),2)
r.saveAsTextFile("C:/BigData/Nume")

In [14]:
r = sc.parallelize([1,2,3,4])
r2 = r.map(lambda x: x + 1)
r2.collect()

[2, 3, 4, 5]

In [15]:
def increment(x):
    return x + 1

r = sc.parallelize([1,2,3,4])
r2 = r.map(increment)
r2.collect()

[2, 3, 4, 5]

In [ ]:
r = sc.parallelize(["hola","hi","ciao")
r2 = r.map(lambda x : len(x))
r2.collect()

In [4]:
import csv
r = sc.parallelize(["1,5,7","8,2,4"])



SyntaxError: invalid syntax (Temp/ipykernel_17000/2794143726.py, line 4)

In [ ]:
r3=r2.map(lambda l: [int](e) for in l)
r3.collect()

In [5]:
r = sc.parallelize(["1,5,7"],["8,2,4"])
re = r.flatMap(lambda s: list(csv.reader[s])[0])
r2.collect()

#

def es_primo(x):
    for i in range(2,x):
        if x % i == 0:
            return False
        return True
r = sc.parallelize(range(2,31))
r2 = r.filter(es_primo)
r2.collect

#

r = sc.parallelize([("a",0),("b"1),("c",2)])
r2 = r.mapValues(lambda x: x+1)
r2.collect()

#

import csv
r = sc.parallelize([("a","1,5,7"),("b","8,2")])
r2 = r.flatMapValues(lambda x: list(csv.reader([x]))[0])
re.collect()

#

r = sc.parallelize([("a",3.14),("b",9.4),("a",2.7)])
r2 = r.groupByKey()
r2.collect()

#

r3 = r2.mapValues(lambda x: list(x))
r3.collect()

#

r = sc.parallelize([("a",2),("b",1),("a",3)])
r2 = r.reduceByKey(lambda x,y: x+y)
r2.collect()

#

r1 = sc.parallelize([1,2,3,4])
r2 = sc.parallelize([2,4,6])
r3 = r1.union(r2)
r3.collect()

#

r4 = r3.distinct()
r4.collect()

#

r1 = sc.parallelize([1,2,3,4])
r2 = sc.parallelize([2,4,2,6])
r3 = r1.union(r2)
r3.collect()

#

r1 = sc.parallelize([1,2,3,4,1])
r2 = sc.parallelize([2,6])
r3 = r1.subtract(r2)
r3.collect()

#

r1 = sc.parallelize([1,2,3])
r2 = sc.parallelize(["a","b"])
r3 = r1.cartesian(r2)
r3.collect()

#

r1 = sc.parallelize([("a",1),("b",2),("c",3)])
r2 = sc.parallelize([("b",8),("d",7)])
r3 = r1.join(r2)
r3.collect()

#

r1 = sc.parallelize([("a",1),("b",2),("c",3)])
r2 = sc.parallelize([("b",8),("d",7),("b",0)])
r3 = r1.join(r2)
r3.collect()

#

sc.stop()

SyntaxError: invalid syntax (Temp/ipykernel_17000/1325151276.py, line 18)

In [6]:
sc.stop()